In [ ]:
%cd ../code_
from collections import Counter
from tableTestingNewMacro import *
from utils import Convert_back_to_df
from compare import compare_per_cell
import pandas as pd
import os
import ast
import json
from typing import Any, List
from itertools import combinations
import tqdm


In [ ]:
def check(expected_df: pd.DataFrame, answer: pd.DataFrame) -> bool:
    expected_df = expected_df.astype(str)
    try:
        if expected_df.shape == answer.shape:
            return expected_df.equals(answer)
        else:
            return False
    except:
        return False


def metric_pass_k(expected_df: pd.DataFrame, answer: List[pd.DataFrame], k: int) -> float:
    expected_df = expected_df.astype(str)
    boolean_answers = [check(expected_df, answer[i])
                       for i in range(len(answer))]
    combinations_k = list(combinations(boolean_answers, k))
    passed_at_k = 0
    # Calculate the pass@k metric
    for comb in combinations_k:
        if any(comb):
            passed_at_k += 1
    pass_at_k_percentage = (passed_at_k / len(combinations_k))*100

    return pass_at_k_percentage

In [ ]:
def serialize_it(expected_df, format_name):
    if format_name == "JsonFormat":
        return str(JsonFormat().formatting(next(SerializeTable().modify(expected_df))))
    if format_name == "DFloaderFormat":
        return str(DFloaderFormat().formatting(next(SerializeTable().modify(expected_df))))
    if format_name == "DataMatrixFormat":
        return str(DataMatrixFormat().formatting(next(SerializeTable().modify(expected_df))))
    if format_name == "MarkdownFormat":
        return str(MarkdownFormat().formatting(next(SerializeTable().modify(expected_df))))
    if format_name == "CommaSeparatedFormat":
        return str(CommaSeparatedFormat().formatting(next(SerializeTable().modify(expected_df))))
    if format_name == "TabSeparatedFormat":
        return str(TabSeparatedFormat().formatting(next(SerializeTable().modify(expected_df))))

In [ ]:
benchmark_data_path = "../resources/BenchmarkDataset"
benchmark_dataset = os.listdir(benchmark_data_path)
# 3----> macro test -noises 8 formats 3 test
benchmark_save_path = "../resources/BenchmarkResults_29_9_2023_Macro_Tests_all_noise_redo_redo/"
benchmark_dataset = [d for d in benchmark_dataset if d.endswith(".csv")]
sp = spa = benchmark_save_path

for bd in benchmark_dataset:
    try:
        pass_metrics = []
        t_F_bools = []
        scores_all_precision = []
        scores_all_recall = []
        print(bd)
        df = pd.read_csv(os.path.join(benchmark_save_path,
                         bd, "Macro_"+bd+"_output.csv"))
        print(df.shape)
        dirs = os.listdir(os.path.join(
            benchmark_save_path, bd, "cache_logger_new"))
        print(dirs)
        error_lines = 0
        error_converts = 0
        found = 0
        table_ops = []
        error_converts_format = []
        ops_candidate_error = []
        test_candidate_error = []
        error_logs = []
        print("use:", dirs[-1])
        with open(os.path.join(benchmark_save_path, bd, "cache_logger_new", dirs[-1]), "r") as file:
            for line in tqdm.tqdm(file):
                try:
                    data_ = json.loads((line))
                    expected_ans = data_["test-expected-answer"]
                    expected_df = Convert_back_to_df(
                        "JsonFormat", str(expected_ans))
                    gen = data_["gen"]
                    if gen == "TableReconstructionTests" and data_["table_op"] == "SampleRows":
                        if serialize_it(expected_df, data_["tab_format"]) in data_["prompt_cache_per_call"]["prompt"]:
                            table_op = "SerializeTable"
                            found += 1
                    elif gen == "TableReconstructionTests1":
                        table_op = "SerializeTable"
                    else:
                        table_op = data_["table_op"]
                    table_ops.append(table_op)
                    answer = data_["answer"]
                    tab_format = data_["tab_format"]
                    answer_changed_format = []
                    for a in answer:
                        try:
                            conv = Convert_back_to_df(tab_format, a)
                        except Exception as err:
                            conv = pd.DataFrame()
                            error_converts += 1
                            if a == answer[0]:
                                error_converts_format.append(
                                    data_["tab_format"])

                        answer_changed_format.append(conv)

                    a = answer_changed_format[0]
                    try:
                        scores_all_precision.append(
                            compare_per_cell(a, expected_df)*100)
                    except:
                        scores_all_precision.append(0)
                    try:
                        # , type_reference_df=a, count_header_and_index=True, return_fraction=False))
                        scores_all_recall.append(
                            compare_per_cell(expected_df, a)*100)
                    except:
                        scores_all_recall.append(0)

                    # break
                    try:
                        pass_metrics.append([metric_pass_k(
                            expected_ans, answer_changed_format, k) for k in [1, 3, 5, 10, 15]])
                    except Exception as Err:
                        pass_metrics.append([None for k in [1, 3, 5, 10, 15]])
                    try:
                        t_F_bools.append(
                            check(expected_ans, answer_changed_format[0]))

                    except Exception as Err:
                        t_F_bools.append(False)
                except Exception as err:

                    ops_candidate_error.append(data_["table_op"])
                    test_candidate_error.append(data_["gen"])
                    error_logs.append(data_["error"])
                    error_lines += 1
                    scores_all_precision.append(0)
                    scores_all_recall.append(0)
                    t_F_bools.append(False)
                    pass_metrics.append([None for k in [1, 3, 5, 10, 15]])

        print(len(pass_metrics))
        print(Counter(ops_candidate_error).items())
        print(Counter(test_candidate_error).items())
        print(Counter(error_logs).items())
        print("NO of error lines:", error_lines)
        print("error_converts:", error_converts/15, error_converts)
        print("found serialized:", found)
        print(Counter(table_ops).items())
        print("failed format counts:", Counter(error_converts_format).items())
        df["table_ops"] = table_ops
        df[['pass_1', 'pass_3', 'pass_5', 'pass_10', 'pass_15']] = pass_metrics
        df["Result"] = df["Result"].apply(lambda x: 1 if x else 0)
        df["T/F"] = t_F_bools
        df["precision_per_cell_correctness_top1"] = scores_all_precision
        df["recall_per_cell_correctness_top1"] = scores_all_recall
        df.to_csv(os.path.join(benchmark_save_path, bd,
                  "Macro_"+bd+"_output_revamped.csv"), index=None)

        print(f"{'-'*90}\n")
    except Exception as Err:
        print("error encountered")
        print(Err)
        pass
    print(f"{'-'*90}\n")